Euclid GC Fisher Matrix Code

Authors: Alkistis Pourtsidou and Dida Markovic, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

Using CAMB installed at /Users/alkistis/workspace


In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Flag to read
fromfile = True

#path flag
SlavMonkey = False

if SlavMonkey:
    path="/Users/dida/Google Drive/Euclid/IST GC/Inputs/"
else:
    path="/Users/alkistis/Google Drive/IST GC/Inputs/"

pathrt = path + "IST_Pks8sqRatios_plmn_z/"
pathfg = path
pathPk = path + "IST_Pk_plmn_z/"

In [5]:
#Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

gamma=0.545

c=3e5

#print om0

In [6]:
if not fromfile:
    #Set up the fiducial cosmology
    pars = camb.CAMBparams()
    #Set cosmology
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [7]:
if not fromfile:
    #calculate results for these parameters
    results = camb.get_results(pars)
    #print pars

In [8]:
if not fromfile:
    #Get matter power spectrum at z=0: P(k,z=0)

    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)

    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

In [9]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

#Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

#print rcom(ztest)
#print DA(ztest)

In [10]:
#LCDM growth rate and growth factor
if not fromfile:

    def fg(zz):
        omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
        return pow(omz,gamma)

    def Dg_dz(zz):
        return -fg(zz)/(1+zz)

    def Dgz(zc):
        start_z = 0.0
        ans = scipy.integrate.romberg(Dg_dz, start_z, zc)
        return np.exp(ans)
    
else:

    growthfilename = pathfg+"/IST_zetha_fg_growthfactor.dat"
    datafg = np.genfromtxt(growthfilename)
    
    zg = datafg[:,0]
    fg = datafg[:,1]
    Dg = datafg[:,2]
    
    def fg(zz, zi=zg, fi=fg):
        ans = interp1d(zi, fi)
        return ans(zz)

    def Dgz(zz, zi=zg, di=Dg):
        ans = interp1d(zi, di)
        return ans(zz)
    
print fg(0.7), fg(1.7)

0.821175740046 0.945876313858


In [11]:
filename = pathrt+"/Pks8sqRatio_ist_fid_iz=00.dat"
 
if fromfile:
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1] # This file is not z=0!! It is z=0.7, so we need to correct later if z=0 is needed!
    sig8_fid = data[0,2]
else:
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

#print results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)[2]

In [12]:
#Construct P(k,z=0) interpolating function
if not fromfile:
    Pkz0 = interp1d(kh, pk[0])

    print Pkz0(0.1)

In [13]:
#get fiducial σ8 (at z=0)
if not fromfile:
    sig8_fid = results.get_sigma8()[0]
    print sig8_fid

    rt0_fid = interp1d(kh, pk[0]/pow(sig8_fid,2))
else:
    rt0_fid = interp1d(kh, rt)
    
    Pkz0 = interp1d(kh, rt*pow(sig8_fid,2)/pow(Dgz(0.7),2))

In [14]:
#Redshift bins
zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001
kmax = 0.2

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
14 14


In [15]:
#Construct P_gg(k,μ,z)
def Pgg(kk,mu,zc):
    return pow(bg+fg(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)    

In [16]:
#Euclid
Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

Dzbin = 0.1

In [17]:
def photoz(kk,mu,zc): #set to zero!
    sigz = 0.00*(1+zc) 
    return np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))

print photoz(0.5,0.5,ztest)

1.0


In [18]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return Vsur(zc)/Ngal

#effective volume; the hubble**3 factors are for units consistency
def Veff(kk,mu,zc):
    return hubble**3*Vsur(zc)*(Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc)\
                                                     +hubble**3*Pshot(zc)))**2

print ztest, "%.4g" %  Vsur(ztest)

0.7 9.184e+09


In [19]:
#Fisher matrix parameters 
params = ["0:lnH","1:lnDA","2:lnfsig8","3:lnbsig8","4:Ps","5:ns","6:wb","7:wm","8:h"]
shape_params = [5,6,7,8]

In [20]:
#Fisher matrix derivatives 

#z-dependent parameters

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)/(bg+mu**2*fg(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bg/(bg+mu**2*fg(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax-kmin)/400
    return -2.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax-kmin)/400
    return 1.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dPs(kk,mu,zc):
    return 1.0/(Pgg(kk,mu,zc)*photoz(kk,mu,zc))

In [21]:
#shape parameters

if not fromfile:
    # Numerical derivative wrt ns 

    step_ns = 0.01*nss
    ns_p = nss+step_ns
    ns_m = nss-step_ns

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=ns_p, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_ns_p = results.get_sigma8()[0]
    #Pkz0_ns_p = interp1d(kh, pk[0])
    
    rt0_ns_p = interp1d(kh,pk[0]/pow(sig8_ns_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=ns_m, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_ns_m= results.get_sigma8()[0]
    #Pkz0_ns_m = interp1d(kh, pk[0])
    
    rt0_ns_m = interp1d(kh,pk[0]/pow(sig8_ns_m,2))

    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    #print sig8_ns_m, sig8_fid, sig8_ns_p
    #print results.get_sigma8()[0]
    
    # Numerical derivative wrt wb 

    wb = omegab*pow(hubble,2)
    wc = omegac*pow(hubble,2)
    step_wb = 0.01*wb
    wb_p = wb+step_wb
    wb_m = wb-step_wb
    wc_p = wc-step_wb
    wc_m = wc+step_wb

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb_p, omch2=wc_p,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wb_p = results.get_sigma8()[0]
    #Pkz0_wb_p = interp1d(kh, pk[0])

    rt0_wb_p = interp1d(kh,pk[0]/pow(sig8_wb_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb_m, omch2=wc_m,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wb_m= results.get_sigma8()[0]
    #Pkz0_wb_m = interp1d(kh, pk[0])

    rt0_wb_m = interp1d(kh,pk[0]/pow(sig8_wb_m,2))

    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    #print sig8_wb_m, sig8_fid, sig8_wb_p
    #print results.get_sigma8()[0]

    # Numerical derivative wrt wm 

    wm = (omegab+omegac)*pow(hubble,2)
    step_wm = 0.01*wm
    wm_p = wm+step_wm
    wm_m = wm-step_wm

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb, omch2=wm_p-wb,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wm_p = results.get_sigma8()[0]
    #Pkz0_wm_p = interp1d(kh, pk[0])

    rt0_wm_p = interp1d(kh,pk[0]/pow(sig8_wm_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=wb, omch2=wm_m-wb,omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    sig8_wm_m= results.get_sigma8()[0]
    #Pkz0_wm_m = interp1d(kh, pk[0])

    rt0_wm_m = interp1d(kh,pk[0]/pow(sig8_wm_m,2))

    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    #print sig8_wm_m, sig8_fid, sig8_wm_p
    #print results.get_sigma8()[0]

    # Numerical derivative wrt h 

    step_h = 0.01*hubble
    h_p = hubble+step_h
    h_m = hubble-step_h

    #DERIV PLUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h_p, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    #print pars

    sig8_h_p = results.get_sigma8()[0]
    #Pkz0_h_p = interp1d(kh, pk[0])

    rt0_h_p = interp1d(kh,pk[0]/pow(sig8_h_p,2))

    #DERIV MINUS
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h_m, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 800)

    #print pars

    sig8_h_m= results.get_sigma8()[0]
    #Pkz0_h_m = interp1d(kh, pk[0])

    rt0_h_m = interp1d(kh,pk[0]/pow(sig8_h_m,2))


    #RESET FIDUCIAL
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
    pars.set_dark_energy() #LCDM (default)
    pars.InitPower.set_params(ns=nss, r=0, As=Ass)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    results = camb.get_results(pars)
    #Not non-linear corrections couples to smaller scales than you want
    pars.set_matter_power(redshifts=[0.], kmax=2.0)
    #Just linear spectra
    pars.NonLinear = model.NonLinear_none
    results.calc_power_spectra(pars)

    print sig8_h_m, sig8_fid, sig8_h_p
    print results.get_sigma8()[0]

In [22]:
if fromfile:
    
    
    ############ Numerical derivative wrt ns

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=03_iz=00.dat"
    
    # Get the value of ns_p
    ns_p = 9.696e-01 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_ns_p = interp1d(kh,rt)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=03_iz=00.dat"
    
    # Get the value of ns_m
    ns_m = 9.504e-01 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_ns_m = interp1d(kh,rt)

    # Get the half-step
    step_ns = (ns_p - ns_m)/2
    
    
    ############ Numerical derivative wrt wb

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=01_iz=00.dat"
    
    # Get the value of wb_p
    wb_p = 2.26695e-02 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wb_p = interp1d(kh,rt)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=01_iz=00.dat"
    
    # Get the value of wb_m
    wb_m = 2.22206e-02 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wb_m = interp1d(kh,rt)

    # Get the half-step
    step_wb = (wb_p - wb_m)/2


    ############ Numerical derivative wrt wm

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=00_iz=00.dat"
    
    # Get the value of wm_p
    wm_p = 1.45084e-01 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wm_p = interp1d(kh,rt)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=00_iz=00.dat"
    
    # Get the value of wm_m
    wm_m = 1.42212e-01 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_wm_m = interp1d(kh,rt)

    # Get the half-step
    step_wm = (wm_p - wm_m)/2


    ############ Numerical derivative wrt h

    #DERIV PLUS
    filename = pathrt+"Pks8sqRatio_ist_pl_ip=02_iz=00.dat"
    
    # Get the value of h_p
    h_p = 6.76700e-01 # see Melita's file header            

    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_h_p = interp1d(kh,rt)
    
    #DERIV MINUS
    filename = pathrt+"Pks8sqRatio_ist_mn_ip=02_iz=00.dat"
    
    # Get the value of h_m
    h_m = 6.63300e-01 # see Melita's file header
    
    # Get the data from the file
    data = np.genfromtxt(filename, skip_header=3)
    kh = data[:,0]
    rt = data[:,1]
    s = data[0,2]
    rt0_h_m = interp1d(kh,rt)

    # Get the half-step
    step_h = (h_p - h_m)/2

In [23]:
def dlnP_dns(kk):
    #return ((Pkz0_ns_p(kk)/pow(sig8_ns_p,2))-(Pkz0_ns_m(kk)/pow(sig8_ns_m,2)))/(2*step_ns)/(Pkz0(kk)/pow(sig8_fid,2))
    return (rt0_ns_p(kk)-rt0_ns_m(kk))/(2*step_ns)/rt0_fid(kk)
    
def dlnP_dwb(kk):
    #return ((Pkz0_wb_p(kk)/pow(sig8_wb_p,2))-(Pkz0_wb_m(kk)/pow(sig8_wb_m,2)))/(2*step_wb)/(Pkz0(kk)/pow(sig8_fid,2))
    return (rt0_wb_p(kk)-rt0_wb_m(kk))/(2*step_wb)/rt0_fid(kk)
    
def dlnP_dwm(kk):
    #return ((Pkz0_wm_p(kk)/pow(sig8_wm_p,2))-(Pkz0_wm_m(kk)/pow(sig8_wm_m,2)))/(2*step_wm)/(Pkz0(kk)/pow(sig8_fid,2))
    return (rt0_wm_p(kk)-rt0_wm_m(kk))/(2*step_wm)/rt0_fid(kk)
    
def dlnP_dh(kk):
    #return ((Pkz0_h_p(kk)/pow(sig8_h_p,2))-(Pkz0_h_m(kk)/pow(sig8_h_m,2)))/(2*step_h)/(Pkz0(kk)/pow(sig8_fid,2))
    return (rt0_h_p(kk)-rt0_h_m(kk))/(2*step_h)/rt0_fid(kk)
    
print dlnP_dns(0.1), dlnP_dwb(0.1), dlnP_dwm(0.1), dlnP_dh(0.1)
#print dlnP_dwm(0.1), dlnP_dh(0.1)

-0.458617307209 1.84905736371 -0.71818417615 -0.194940813117


In [24]:
#with open('test.txt', 'w') as Pkfile:
#    for row in zip(kh, Pkz0_wm_m(kh)/pow(sig8_wm_m,2)):
#        print >> Pkfile, row[0], row[1]

In [25]:
def dF(kk,mu,zc):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [26]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.romb(dfun.T[i], dx=np.diff(mugrid)[0]) for i in range(kgrid.size)]
    return scipy.integrate.romb(muint, dx=np.diff(kgrid)[0])

In [27]:
mugrid = np.linspace(-1., 1., 513) 

In [28]:
#%%time
#   Fisher matrix   # 

Npar = len(params)
Npar_shape = len(shape_params)
#create array of zeros
s = (Npar-Npar_shape)*Nzbins + Npar_shape

Fishermat = np.zeros([s,s])

kgrid = np.linspace(kmin, kmax, 513)
K, MU = np.meshgrid(kgrid, mugrid)
for zi in range(0,Nzbins):
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
    Ngal = dn3[zi]*Area*Dzbin

    for i in range(0,Npar):
        if i not in shape_params: 
            k=zi*(Npar-Npar_shape) + i
        else:
            k = s + (i - Npar)
            
        def deriv_i(kk,mu,z):
            if i==0:  return dlnP_dlnH(kk,mu,z)
            elif i==1:  return dlnP_dlnDA(kk,mu,z)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,z)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,z)
            elif i==4:  return dlnP_dPs(kk,mu,z)
            elif i==5:  return dlnP_dns(kk)
            elif i==6:  return dlnP_dwb(kk)
            elif i==7:  return dlnP_dwm(kk)
            elif i==8:  return dlnP_dh(kk)
            else: print "Error: index out of range"
        
        for  j in range(0,Npar):
            if j not in shape_params:
                l=zi*(Npar-Npar_shape) + j
            else:
                l = s + (j - Npar)
            
            
            if j>=i:
                def deriv_j(kk,mu,z):
                    if j==0:  return dlnP_dlnH(kk,mu,z)
                    elif j==1:  return dlnP_dlnDA(kk,mu,z)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,z)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,z)
                    elif j==4:  return dlnP_dPs(kk,mu,z)
                    elif j==5:  return dlnP_dns(kk)
                    elif j==6:  return dlnP_dwb(kk)
                    elif j==7:  return dlnP_dwm(kk)
                    elif j==8:  return dlnP_dh(kk)
                    else: print "Error: index out of range" 
                    
            #print zc, params[i], params[j], '[', k, ',', l, ']'
           
            if l>=k: 
                Fishermat[k][l] += integrate2D(dF(K,MU,zc),kgrid,mugrid)
            else: 
                Fishermat[k,l] = Fishermat[l,k]                 

In [29]:
header = 'lnH_0.7 lnDa_0.7 lnfs8_0.7 lnbs8_0.7 Ps_0.7 lnH_0.8 lnDa_0.8 lnfs8_0.8 lnbs8_0.8 Ps_0.8 lnH_0.9 lnDa_0.9 lnfs8_0.9 lnbs8_0.9 Ps_0.9 lnH_1.0 lnDa_1.0 lnfs8_1.0 lnbs8_1.0 Ps_1.0 lnH_1.1 lnDa_1.1 lnfs8_1.1 lnbs8_1.1 Ps_1.1 lnH_1.2 lnDa_1.2 lnfs8_1.2 lnbs8_1.2 Ps_1.2 lnH_1.3 lnDa_1.3 lnfs8_1.3 lnbs8_1.3 Ps_1.3 lnH_1.4 lnDa_1.4 lnfs8_1.4 lnbs8_1.4 Ps_1.4 lnH_1.5 lnDa_1.5 lnfs8_1.5 lnbs8_1.5 Ps_1.5 lnH_1.6 lnDa_1.6 lnfs8_1.6 lnbs8_1.6 Ps_1.6 lnH_1.7 lnDa_1.7 lnfs8_1.7 lnbs8_1.7 Ps_1.7 lnH_1.8 lnDa_1.8 lnfs8_1.8 lnbs8_1.8 Ps_1.8 lnH_1.9 lnDa_1.9 lnfs8_1.9 lnbs8_1.9 Ps_1.9 lnH_2.0 lnDa_2.0 lnfs8_2.0 lnbs8_2.0 Ps_2.0 ns wb wm h'

if fromfile:
    newfile = 'alkistis-case5-with-files.txt'
else:
    newfile = 'alkistis-case5.txt'
np.savetxt(newfile, Fishermat, header=header)
print np.sqrt(np.diag(linalg.inv(Fishermat)))

[  2.59607135e-02   2.53921143e-02   2.22669900e-02   1.93500796e-02
   2.25813345e+01   2.55691692e-02   2.50890810e-02   2.11050384e-02
   1.88165578e-02   1.96488253e+01   2.54351620e-02   2.50235546e-02
   2.05199197e-02   1.87123723e-02   1.70458576e+01   2.53647839e-02
   2.49850297e-02   2.03556954e-02   1.86328962e-02   1.59785402e+01
   2.53360905e-02   2.49557956e-02   2.05791810e-02   1.85430484e-02
   1.63878521e+01   2.53287473e-02   2.49578021e-02   2.06802626e-02
   1.85297135e-02   1.59407565e+01   2.53385553e-02   2.49701238e-02
   2.08656806e-02   1.85324974e-02   1.56692169e+01   2.54051231e-02
   2.50395217e-02   2.11539735e-02   1.86406268e-02   1.52511052e+01
   2.55175243e-02   2.51269635e-02   2.19339073e-02   1.87475731e-02
   1.61260841e+01   2.57416493e-02   2.52867377e-02   2.36442585e-02
   1.89319349e-02   1.86961557e+01   2.62968767e-02   2.58032990e-02
   2.59606726e-02   1.97082569e-02   2.03539038e+01   2.64645736e-02
   2.59667511e-02   2.66499836e-02

In [30]:
#testing